In [1]:
import nltk
import string 
import numpy as np
import pandas as pd
import pathlib
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import PIL
from tensorflow import keras
import random
from keras.models import load_model

# SKIN DISEASE PREDICTION CHATBOT

In [2]:
#our contribution
mapping = ['ACK','BCC','MEL','NEV','SEK','SCC']
full_form = {'ACK':'actinic keratosis', 'BCC':'basal cell carcinoma', 'MEL':'melanoma', 'NEV':'nevus', 'SEK' :'seborrheic keratosis','SCC':'squamous cell carcinoma'}
THRESHOLD = 0.5

In [3]:
#loading the cnn model
cnn_model = load_model("final_model/cnn_sigmoid_model.h5")

In [4]:
import math

class DecisionTree:
    #our contribution
    def read_data(self, filename, l):
        fid = open(filename, "r")
        data = []
        d = []
        temp_d = []
        for line in fid.readlines():
            d.append(line.strip())
        for d1 in d:   
            temp_d.append(d1.split(","))
        fid.close()
        for d2 in temp_d:
            if d2[-1] in l or d2[-1] =='diagnostic':
                data.append(d2)
        self.featureNames = self.get_features(data)
        data = data[1:]
        self.classes = self.get_classes(data)
        data = self.get_pure_data(data)

        return data, self.classes, self.featureNames

    def get_classes(self, data):
        data = data[1:]
        classes = []
        for d in range(len(data)):
            classes.append(data[d][-1])

        return classes

    def get_features(self, data):
        features = data[0]
        features = features[:-1]
        return features

    def get_pure_data(self, dataRows):
        dataRows = dataRows[1:]
        for d in range(len(dataRows)):
            dataRows[d] = dataRows[d][:-1]
        return dataRows

    def zeroList(self, size):
        d = []
        for i in range(size):
            d.append(0)
        return d

    def getArgmax(self, arr):
        m = max(arr)
        ix = arr.index(m)
        return ix

    def getDistinctValues(self, dataList):
        distinctValues = []
        for item in dataList:
            if (distinctValues.count(item) == 0):
                distinctValues.append(item)
        return distinctValues

    def getDistinctValuesFromTable(self, dataTable, column):
        distinctValues = []
        for row in dataTable:
            if (distinctValues.count(row[column]) == 0):
                distinctValues.append(row[column])
        return distinctValues

    def getEntropy(self, p):
        if (p != 0):
            return -p * math.log2(p)
        else:
            return 0

    def create_tree(self, trainingData, classes, features, maxlevel=-1, level=0):
        nData = len(trainingData)
        nFeatures = len(features)

        try:
            self.featureNames
        except:
            self.featureNames = features

        newClasses = self.getDistinctValues(classes)
        frequency = self.zeroList(len(newClasses))
        totalEntropy = 0
        index = 0
        for aclass in newClasses:
            frequency[index] = classes.count(aclass)
            prob = float(frequency[index])/nData
            totalEntropy += self.getEntropy(prob)
            index += 1

        default = classes[self.getArgmax(frequency)]
        if (nData == 0 or nFeatures == 0 or (maxlevel >= 0 and level > maxlevel)):
            return default
        elif classes.count(classes[0]) == nData:
            return classes[0]
        else:
            gain = self.zeroList(nFeatures)
            for feature in range(nFeatures):
                g = self.getGain(trainingData, classes, feature)
                gain[feature] = totalEntropy - g

            bestFeature = self.getArgmax(gain)
            newTree = {features[bestFeature]: {}}

            values = self.getDistinctValuesFromTable(
                trainingData, bestFeature)
            for value in values:
                newdata = []
                newClasses = []
                index = 0
                for row in trainingData:
                    if row[bestFeature] == value:
                        if bestFeature == 0:
                            newRow = row[1:]
                            newNames = features[1:]
                        elif bestFeature == nFeatures:
                            newRow = row[:-1]
                            newNames = features[:-1]
                        else:
                            newRow = row[:bestFeature]
                            newRow.extend(row[bestFeature + 1:])
                            newNames = features[:bestFeature]
                            newNames.extend(features[bestFeature+1:])
                        newdata.append(newRow)
                        newClasses.append(classes[index])
                    index += 1

                subtree = self.create_tree(
                    newdata, newClasses, newNames, maxlevel, level + 1)

                newTree[features[bestFeature]][value] = subtree
            return newTree

        print(newClasses)

    def getGain(self, data, classes, feature):
        gain = 0
        ndata = len(data)

        values = self.getDistinctValuesFromTable(data, feature)
        featureCounts = self.zeroList(len(values))
        entropy = self.zeroList(len(values))
        valueIndex = 0
        for value in values:
            dataIndex = 0
            newClasses = []
            for row in data:
                if row[feature] == value:
                    featureCounts[valueIndex] += 1
                    newClasses.append(classes[dataIndex])
                dataIndex += 1

            classValues = self.getDistinctValues(newClasses)
            classCounts = self.zeroList(len(classValues))
            classIndex = 0
            for classValue in classValues:
                for aclass in newClasses:
                    if aclass == classValue:
                        classCounts[classIndex] += 1
                classIndex += 1

            for classIndex in range(len(classValues)):
                pr = float(classCounts[classIndex])/sum(classCounts)
                entropy[valueIndex] += self.getEntropy(pr)

            pn = float(featureCounts[valueIndex])/ndata
            gain = gain + pn * entropy[valueIndex]

            valueIndex += 1
        return gain

    def showTree(self, dic, seperator):
        if (type(dic) == dict):
            for item in dic.items():
                print(seperator, item[0])
                self.showTree(item[1], seperator + " | ")
        else:
            print(seperator + " -> (", dic + ")")

In [5]:
#our contribution
ques_bank= { 'smoke': 'How often do you smoke?',
          'drink' : 'How often do you drink?',
          'itch' : 'Is your skin itchy?' ,
          'grew' : 'Does your skin disease keep growing?',
          'hurt' : 'Does it hurt?',
          'changed' : 'Has ur skin disease changed?',
          'bleed' : 'Does it bleed?',
          'elevation': 'Is there any elevation?'} 

In [6]:
#our contribution
#function for finding the diseases above the threshold
def shortlist_class(pred):  
    shortlist=[]
    l=list(pred)
    #print('Possible classes:')
    for i in l:
        if i > THRESHOLD:
            #print('class : ',train_ds.class_names[l.index(i)])
            shortlist.append(mapping[l.index(i)])
    return shortlist

In [7]:
#our contribution
def find_shortlist(path1): 
    image1 = tf.keras.utils.load_img(path1,target_size=(180, 180))
    #plt.figure(figsize=(2, 2))
    #plt.imshow(image1)
    #plt.title('input image:')
    #plt.axis("off")
    input_arr = tf.keras.utils.img_to_array(image1)
    input_arr = np.array([input_arr])  
    predictions = cnn_model.predict(input_arr)
    shortlisted_diseases = shortlist_class(predictions[0])
    #print('\nSHORLISTED DISEASES LIST : \n',shortlisted_diseases)
    return shortlisted_diseases

In [8]:
#our contribution
def generate_dynamic_tree(shortlist):
    tree = DecisionTree()
    tr_data, clss, attrs = tree.read_data('datasets/Features.csv',shortlist)
    tree1 = tree.create_tree(tr_data, clss, attrs)
    #tree.showTree(tree1, ' ')
    return tree1

In [9]:
#our contribution
def get_ans(l):
    while(True):
        ans=input('enter option:')
        ans=ans.lower()
        ans=ans.capitalize()
        if ans in l:           
            return ans
        else:
            print('Invalid Input!Try again')

In [10]:
#our contribution
def generate_questionnaire(tree1):
    i=0
    firstnode=1
    while i == 0:
        flag = 1
        for node in tree1.keys():
            tree2=tree1[node]
            if firstnode == 0 and len(tree2.keys())==1:
                flag=0
                break
            print('\n')
            print(ques_bank[node])
            print('opts:')
            l=[]
            for opt in tree2.keys():
                print(opt)
                l.append(opt)
            ans= get_ans(l)
            
        if flag == 1:
            tree1=tree2[ans]
        elif flag ==0 :
            l=list(tree2.keys())
            l=l[0]
            tree1=tree2[l]
        flag = 1
        firstnode=0
        if type(tree1) == str:
            i=1

    print('\nTHE SKIN DISEASE THAT YOU SEEM TO HAVE IS: ', tree1)

In [11]:
#our contribution
def start_sd_Prediction():
    print("\nBOT :Hello,I will be predicting your skin disease..")
    shortlist=[]
    while(True):
        try:
            print("Please provide an image of your skin disease")
            path=input('USER: ')
            if path[0] == 'd':
                shortlist = []
                break
            shortlist= find_shortlist(path)
            #print(shortlist)
        except:
            print("File not found")
            continue
        else:
            break
    
    if len(shortlist) == 0 :   
        print('\nYour skin seems to have no problem!')
    else: 
        if len(shortlist) == 1 :
            print('\nTHE SKIN DISEASE THAT YOU SEEM TO HAVE IS: ', shortlist[0])  
        else:
            tree=generate_dynamic_tree(shortlist)
            print('\n\nBOT : Your image has been analysed!')
            print('Please answer the further questions to make accurate predictions-')
            generate_questionnaire(tree)

In [12]:
#example image
path1='skin-cancer/Skin cancer ISIC The International Skin Imaging Collaboration/Test/melanoma/ISIC_0000031.jpg'
path2='skin-cancer/Skin cancer ISIC The International Skin Imaging Collaboration/Test/nevus/ISIC_0000000.jpg'

# NLP BASED CHATBOT

In [13]:
nltk.download('punkt') #tokenizer for english
nltk.download('wordnet') #corpora

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### preprocessing corpus

In [14]:
data= pd.read_csv('datasets/corpus_data.csv')

In [15]:
data.head()

,category,ques,ans
0,ACK,Can actinic keratosis spread?,Actinic keratoses can be somewhat unpredictabl...
1,ACK,Does vitamin D help actinic keratosis?,Patients with actinic keratoses of the face wh...
2,ACK,Does coconut oil help actinic keratosis?,The best part about treating keratosis pilaris...
3,ACK,Does Vaseline help actinic keratosis?,Wash around the area with clean water 2 times ...
4,ACK,What vitamin treats actinic keratosis?,Vitamin B3 – what is it and how does it work? ...


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  850 non-null    object
 1   ques      850 non-null    object
 2   ans       840 non-null    object
dtypes: object(3)
memory usage: 20.0+ KB


In [17]:
lemmer = nltk.stem.WordNetLemmatizer()

def lemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

def lemNormalize(text):
    return lemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

In [18]:
remove_punct_dict

{33: None,
 34: None,
 35: None,
 36: None,
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,
 42: None,
 43: None,
 44: None,
 45: None,
 46: None,
 47: None,
 58: None,
 59: None,
 60: None,
 61: None,
 62: None,
 63: None,
 64: None,
 91: None,
 92: None,
 93: None,
 94: None,
 95: None,
 96: None,
 123: None,
 124: None,
 125: None,
 126: None}

In [19]:
#our contribution
cat = ['ACK','BCC','MEL','NEV','SEK','SCC','GEN']
main_list=[]
index_list=[]

for i in cat:
    m_list=[]
    i_list=[]
    for j in data.index:
        if data.category[j]==i:
            m_list.append(data.ques[j].lower())
            i_list.append(j)
    main_list.append(m_list)
    index_list.append(i_list)

In [20]:
main_list[6]

['different types of cancer that start in the skin.',
 'what are the different types of skin cancer?',
 'what is atypical moles?',
 'when should i call or visit doctor?',
 'what are  the abcde criteria?',
 'what does abcde stand for?',
 'what are the most dangerous skin cancers?',
 'what are the features of bazex syndrome?',
 'what is rombo syndrome?',
 'what is mohs surgery?',
 'what is cryosurgery?',
 'what is cryosurgery in actinic keratosis?',
 'what is chemical peel?',
 'what is laser resurfacing?',
 'what is the difference between a mole and a freckle? ',
 'what is cryosurgery?',
 'what is chemical peel?',
 'what is curettage?',
 'what is photodynamic therapy?',
 'what is laser resurfacing?',
 'different types of cancer that start in the skin.',
 'what are the different types of skin cancer?',
 'what are  the abcde criteria?',
 'what does abcde stand for?',
 'what are the most dangerous skin cancers?',
 'what are skin cancers',
 'what can skin cancer do',
 'when should a skin can

In [21]:
data.ques[index_list[6][0]]

'different types of cancer that start in the skin.'

In [22]:
data.ans[index_list[6][0]]

'There are two main forms of skin cancer: melanoma and nonmelanoma. \nMelanoma is a rare form of skin cancer. It is more likely to invade nearby tissues and spread to other parts of the body than other types of skin cancer. When melanoma starts in the skin, it is called cutaneous melanoma. Melanoma may also occur in mucous membranes (thin, moist layers of tissue that cover surfaces such as the lips). This PDQ summary is about cutaneous (skin) melanoma and melanoma that affects the mucous membranes.\nThe most common types of skin cancer are basal cell carcinoma and squamous cell carcinoma. They are nonmelanoma skin cancers. Nonmelanoma skin cancers rarely spread to other parts of the body.'

In [23]:
GREETING_INPUTS = ("hello", "hi", "greetings", "hey",)

GREETING_RESPONSES = ["Hi, how can I help you", "Hey, what is that you are looking for", "Hello, please type in your queries"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
#our contribution - did Changes to existing code
def response(user_response,category):
    bot_response = ''
    sent_tokens=main_list[category]
    
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=lemNormalize, stop_words = 'english')
    
    tfidf = TfidfVec.fit_transform(sent_tokens) 
    
    # Get the cosine similarity between user query and all the question in question data. This will be a vector of similarities
    vals = cosine_similarity(tfidf[-1], tfidf)
   
    # Get index of the sentence which matches the most with the query
    idx = vals.argsort()[0][-2]
    # Get the td-idf value of the index which matched the most
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if(req_tfidf == 0):
        bot_response =  "I am sorry! I don't have a answer for this"
    else:
        #our contribution
        bot_response = data.ans[index_list[category][idx]] 
    sent_tokens.remove(user_response)
    return bot_response


In [26]:
#our contribution
def check_quit():
    print('\nENTER 1 to go to the main menu\nEnter 2 to exit')
    quit=int(input("Enter your option:"))
    if quit==1:
        return True
    elif quit==2:
        print('\nBOT: BYE! Take care')
        return False
    else:
        print('Invalid input! Try Again')
        return check_quit()

In [27]:
#our contribution
def format_user_response(txt):
    if txt[-1]=='?' or txt[-1]=='.':
        txt = txt[:len(txt)-1]
    cat = ['ACK','BCC','MEL','NEV','SEK','SCC']
    full_form = {'ACK':'actinic keratosis', 'BCC':'basal cell carcinoma', 'MEL':'melanoma', 'NEV':'nevus', 'SEK' :'seborrheic keratosis','SCC':'squamous cell carcinoma'}

    txt=txt.upper()
    t=txt.split()
    flag= False
    for i in cat: 
        if i in t:
            t[t.index(i)]=full_form[i]
            category = i
            flag =True
            break
            
    msg =  ' '.join([str(elem) for elem in t])
    msg=msg.lower()
    if flag == True:
        index=cat.index(category)
    else:
        full = ['actinic keratosis', 'basal cell carcinoma', 'melanoma', 'nevus', 'seborrheic keratosis','squamous cell carcinoma']
        for i in full:
            if i in msg:
                category=i
                flag=True
                break
        if flag== True:
            index=full.index(category)
        else:
            index=6
    return msg,index

In [28]:
#our contribution - did changes to existing code
def start_nlp_chatbot():
    flag = True

    print("\nHi. I will answer your queries. Please type in your questions here. If you want to exit, type Bye! ")

    while( flag == True):

        user_response = input('\nUSER:')
        #our contribution
        user_response,category = format_user_response(user_response)

        if(user_response != 'bye'):
            if(user_response == 'thanks' or user_response == 'thank you'):
                flag = False
                print("BOT: You are welcome..")
            else:
                if(greeting(user_response) != None):
                    print("BOT: "+greeting(user_response))
                else:
                    print("BOT: ", end="")
                    print(response(user_response,category))
        else:
            flag = False
            print("\nBOT: Bye! take care..")

In [29]:
#our contribution
def start_bot():
    print('BOT: HEY! I am the chatbot which will help you with skin related diseases')
    print('BOT: Please choose what you need:\n')
    flag=True
    
    while(flag==True):
        print('--MAIN MENU--')
        print('Enter 1 for predicting your skin disease')
        print('Enter 2 for asking queries related to skin diseases')
        print('Enter 3 to quit')
        try:
            opt=int(input("Enter your option:"))
        except ValueError:
            print('\nInvalid input! Try Again\n')    
            continue
        else:
            if opt==1:
                print('\n')
                start_sd_Prediction()
                flag=check_quit()
            elif opt==2:
                start_nlp_chatbot()
                flag=check_quit()
            elif opt==3:
                flag=False
                print('\nBOT: BYE! Take care')
            else:
                print('\nInvalid input! Try Again')
            print('\n')
    

In [30]:
#skin-cancer/Skin cancer ISIC The International Skin Imaging Collaboration/Test/melanoma/ISIC_0000031.jpg
#PAD/IMAGES/PAT_1516_1765_530.png
#PAD/IMAGES/PAT_46_881_939.png
#only one shortlist - PAD/IMAGES/PAT_230_1008_673.png


# start_bot()


In [31]:
start_bot()

BOT: HEY! I am the chatbot which will help you with skin related diseases
BOT: Please choose what you need:

--MAIN MENU--
Enter 1 for predicting your skin disease
Enter 2 for asking queries related to skin diseases
Enter 3 to quit
Enter your option:1



BOT :Hello,I will be predicting your skin disease..
Please provide an image of your skin disease
USER: PAD/IMAGES/PAT_1516_1765_530.png
1/1 [==============================] - 1s 567ms/step


BOT : Your image has been analysed!
Please answer the further questions to make accurate predictions-


Has ur skin disease changed?
opts:
No
Yes
Unknown
enter option:no


How often do you drink?
opts:
Rarely
Never
Frequently
enter option:rarely


Is your skin itchy?
opts:
Yes
No
enter option:yes


Does your skin disease keep growing?
opts:
No
Yes
enter option:no

THE SKIN DISEASE THAT YOU SEEM TO HAVE IS:  SEK

ENTER 1 to go to the main menu
Enter 2 to exit
Enter your option:1


--MAIN MENU--
Enter 1 for predicting your skin disease
Enter 2 for as

In [32]:
start_bot()

BOT: HEY! I am the chatbot which will help you with skin related diseases
BOT: Please choose what you need:

--MAIN MENU--
Enter 1 for predicting your skin disease
Enter 2 for asking queries related to skin diseases
Enter 3 to quit
Enter your option:3

BOT: BYE! Take care




In [33]:
1

1